<a href="https://colab.research.google.com/github/musa1993/B/blob/master/nafkem_Hamoye.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow import keras

In [ ]:
train_classes= pd.read_csv('/content/train_v2.csv')

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
train_classes.describe()

In [ ]:
train_classes.head()

In [ ]:
train_classes.shape

In [ ]:
train_classes.isnull().sum()

In [ ]:
train_classes.info()

In [ ]:
#defining a function to add unique labels to a set
labels = set ()
def splitting_tags(tags):
  for tag in tags.split():
      labels.add(tag)   

In [ ]:
#redefine the train_classes;create a copy to avoid overwritting the existing one.
train_classes1=train_classes.copy()

train_classes1['tags'].apply(splitting_tags)
labels = list(labels)
print(labels)

In [ ]:
train_classes1

In [ ]:
#import tensorflow libraies for training the dataset
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense,BatchNormalization,Conv2D, MaxPooling2D
from tensorflow.keras.layers import Dropout,Flatten
from tensorflow.keras.optimizers import Adam,SGD
from tensorflow.keras.callbacks import EarlyStopping,ModelCheckpoint,TensorBoard
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [ ]:
#assert that the length of the dataframe is the same as the shaoe
assert len(train_classes1['image_name'].unique())==train_classes1.shape[0]

In [ ]:
#adding .jpg extension to the column image_name so as to have same name format as the image files
train_classes1['image_name'].apply(lambda x:'{}.jpg'.format(x))
train_classes1.head()

In [ ]:
##One hot encoding is performed on the labels in train classes
for tag in labels:
    train_classes1[tag] = train_classes1['tags'].apply(lambda x: 1 if tag in x.split() else 0)
    


In [ ]:
## adding .jpg extension to the column image_name so as to have same name format as the image files
train_classes1['image_name'] = train_classes1['image_name'].apply(lambda x: '{}.jpg'.format(x))
train_classes1.head()

In [ ]:
#define a function for accuracy for multi_label classification
def multi_label_acc(y_true, y_pred, epsilon = 1e-4):
    
    y_true = tf.cast(y_true, tf.float32)
    y_pred = tf.cast(tf.greater(tf.cast(y_pred, tf.float32), tf.constant(0.5)), tf.float32)
    
    tp = tf.reduce_sum(y_true * y_pred, axis = 1)
    fp = tf.reduce_sum(y_pred, axis = 1) - tp
    fn = tf.reduce_sum(y_true, axis = 1) - tp
    
    y_true = tf.cast(y_true, tf.bool)
    y_pred = tf.cast(y_pred, tf.bool)
        
    tn = tf.reduce_sum(tf.cast(tf.logical_not(y_true), tf.float32) * tf.cast(tf.logical_not(y_pred), tf.float32), 
                       axis = 1)
    return (tp+tn)/(tp+tn+fp+fn+epsilon)

In [ ]:
#modelcheckpoint is set to monitor the model using validation fbeta score and save the best only
save_best_check_point = ModelCheckpoint(filepath = 'best_model.hdf5', monitor = 'val_fbeta', mode = 'max',save_best_only = True, save_weights_only = True)

In [ ]:
#initializing imagedatagenerator with a validation split of 0.2
train_image_gen = ImageDataGenerator(rescale = 1/255, validation_split = 0.2)


In [ ]:
columns = list(train_classes1. columns [2:])

In [ ]:
columns

In [ ]:
def build_model():
  model = Sequential()
  model.add(BatchNormalization(input_shape=(128, 128,3)))
  model.add(Conv2D(32, kernel_size=(3,3),padding='same', activation='relu'))
  model.add(Conv2D(32, kernel_size=(3,3), activation='relu'))
  model.add(MaxPooling2D(pool_size=(2,2)))
  model.add(Dropout(0.2))

  model.add(Conv2D(64,kernel_size=(3,3),padding='same',activation='relu'))
  model.add(Conv2D(64,kernel_size=(3,3),padding='same',activation='relu'))
  model.add(MaxPooling2D(pool_size=(2,2)))
  model.add(Dropout(0.2))

  model.add(Conv2D(128,kernel_size=(3,3),padding='same',activation='relu'))
  model.add(Conv2D(128,kernel_size=(3,3),padding='same',activation='relu'))
  model.add(MaxPooling2D(pool_size=(2,2)))
  model.add(Dropout(0.2))

  model.add(Conv2D(256,kernel_size=(3,3),padding='same',activation='relu'))
  model.add(Conv2D(256,kernel_size=(3,3),padding='same',activation='relu'))
  model.add(MaxPooling2D(pool_size=(2,2)))
  model.add(Dropout(0.2))


  model.add(Flatten())
  model.add(Dense(512, activation='relu'))
  model.add(Dropout(0.5))
  model.add(Dense(17, activation='sigmoid'))

  opt = Adam(ir=1e-4)

  model.compile(loss='niinary_crossentropy', optimizer=opt, metrics=[multi_labelacc, fbeta])

  return model

In [ ]:
#setting up step size for training and validation image data
step_train_size = int(np.ceil(train_generator.samples / train_generator.batch_size))
step_val_size = int(np.ceil(val_generator.samples / val_generator.batch_size))


In [ ]:
#initializing the model
model1 = build_model()

In [ ]:
#this shows the summary of the model, simply put as the model architecture
model1.summary()

In [ ]:
#fitting our model using the parameters already defined 
model1.fit(x = train_generator, steps_per_epoch = step_train_size, validation_data = val_generator, validation_steps = step_val_size,epochs = 25, 
callbacks = [save_best_check_point])